It is clear that to train this kind of models we need contiguous in time images.
We need to figure out a nice way to do this, we cannot simply take the \(k\) previous images because there are some temporal holes between a rainfall and another.
We could give an id to the storms, divide in sequences the storms, which means that when we meet 2 images that are more than 5 minutes temporally spaced we consider the 2 images belonging to different storms. then, in training and choosing epochs we choose randomly one storm and randomly again we choose \(k\) contiguous images of that storm to predict the \(k\)-th.

In [34]:
import numpy as np
import pandas as pd

In [35]:
id_df = pd.read_csv('id_df_final_10.csv')

## dataset creation

In [36]:
id_df = id_df[['id', 'sequence', 'rain_category']]

In [ ]:
# Create a dataframe with the ids of the images
id_seq_dataset = pd.DataFrame()

k_in = 5 # Number of images to predict the next one
k_out = 5
# Create an empty list to accumulate each sequence's data
data = []

for i in id_df['sequence'].unique():
    print(i)
    seq_id = i
    len_seq = len(id_df[id_df['sequence'] == seq_id])
    frames = id_df[id_df['sequence'] == seq_id]
    
    for j in range(len_seq - k_in - k_out - 1):
        k_frames = frames.iloc[j:j+k_in][['id', 'rain_category']]
        id_frames_to_pred = frames.iloc[j+k_in:j+k_in+k_out][['id', 'rain_category']]
        k_frames = pd.concat([k_frames, id_frames_to_pred], ignore_index=True, axis=0)
        rain_category = np.mean(k_frames['rain_category'])
        ids = [int(id_) for id_ in k_frames['id'].tolist()]
        data.append(ids + [seq_id, rain_category])

id_seq_dataset = pd.DataFrame(data, columns=[f'id_{i}' for i in range(k_in + k_out)] + ['sequence', 'rain_category'])

In [47]:
print(len(id_seq_dataset))
id_seq_dataset

15636


,id_0,id_1,id_2,id_3,id_4,id_5,id_6,id_7,id_8,id_9,sequence,rain_category
0,2455220,2455200,2455180,2455162,2455140,2455120,2455080,2455060,2455040,2455020,0,0.2
1,2455200,2455180,2455162,2455140,2455120,2455080,2455060,2455040,2455020,2455002,0,0.3
2,2455180,2455162,2455140,2455120,2455080,2455060,2455040,2455020,2455002,2454980,0,0.4
3,2455162,2455140,2455120,2455080,2455060,2455040,2455020,2455002,2454980,2454940,0,0.5
4,2455140,2455120,2455080,2455060,2455040,2455020,2455002,2454980,2454940,2454920,0,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...
15631,1944569,1944553,1944535,1944505,1944487,1944469,1944451,1944433,1944415,1944387,170,2.3
15632,1944553,1944535,1944505,1944487,1944469,1944451,1944433,1944415,1944387,1944369,170,2.5
15633,1944535,1944505,1944487,1944469,1944451,1944433,1944415,1944387,1944369,1944353,170,2.6
15634,1944505,1944487,1944469,1944451,1944433,1944415,1944387,1944369,1944353,1944330,170,2.7


In [48]:
# Rename the columns
col_names = [f"input_{i}" for i in range(0, k_in)]
col_names = col_names + [f"output_{i}" for i in range(0, k_out)]
col_names = col_names + ["seq_id"] + ["rain_category"]

id_seq_dataset.columns = col_names
id_seq_dataset

,input_0,input_1,input_2,input_3,input_4,output_0,output_1,output_2,output_3,output_4,seq_id,rain_category
0,2455220,2455200,2455180,2455162,2455140,2455120,2455080,2455060,2455040,2455020,0,0.2
1,2455200,2455180,2455162,2455140,2455120,2455080,2455060,2455040,2455020,2455002,0,0.3
2,2455180,2455162,2455140,2455120,2455080,2455060,2455040,2455020,2455002,2454980,0,0.4
3,2455162,2455140,2455120,2455080,2455060,2455040,2455020,2455002,2454980,2454940,0,0.5
4,2455140,2455120,2455080,2455060,2455040,2455020,2455002,2454980,2454940,2454920,0,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...
15631,1944569,1944553,1944535,1944505,1944487,1944469,1944451,1944433,1944415,1944387,170,2.3
15632,1944553,1944535,1944505,1944487,1944469,1944451,1944433,1944415,1944387,1944369,170,2.5
15633,1944535,1944505,1944487,1944469,1944451,1944433,1944415,1944387,1944369,1944353,170,2.6
15634,1944505,1944487,1944469,1944451,1944433,1944415,1944387,1944369,1944353,1944330,170,2.7


In [49]:
# Filter by the amount of rain
id_seq_dataset = id_seq_dataset[id_seq_dataset['rain_category'] > 3]
id_seq_dataseid = id_seq_dataset.astype(int)
id_seq_dataset

,input_0,input_1,input_2,input_3,input_4,output_0,output_1,output_2,output_3,output_4,seq_id,rain_category
198,2449014,2448994,2448974,2448954,2448934,2448914,2448874,2448856,2448834,2448814,3,3.1
199,2448994,2448974,2448954,2448934,2448914,2448874,2448856,2448834,2448814,2448794,3,3.2
200,2448974,2448954,2448934,2448914,2448874,2448856,2448834,2448814,2448794,2448774,3,3.3
201,2448954,2448934,2448914,2448874,2448856,2448834,2448814,2448794,2448774,2448736,3,3.4
202,2448934,2448914,2448874,2448856,2448834,2448814,2448794,2448774,2448736,2448716,3,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...
15617,1951608,1951588,1951568,1951548,1951528,1951509,1951481,1951463,1951445,1951427,169,3.5
15618,1951588,1951568,1951548,1951528,1951509,1951481,1951463,1951445,1951427,1951409,169,3.4
15619,1951568,1951548,1951528,1951509,1951481,1951463,1951445,1951427,1951409,1951391,169,3.3
15620,1951548,1951528,1951509,1951481,1951463,1951445,1951427,1951409,1951391,1951372,169,3.2


In [50]:
id_seq_dataset.to_csv('../data/id_seq_dataset_10.csv', index=False)